In [1]:
# Useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi


from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer, execute
from qiskit.quantum_info import Pauli, state_fidelity, basis_state, process_fidelity 

from qiskit.circuit import Parameter


from qiskit import *
IBMQ.load_accounts(hub=None)
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.ibmq import least_busy

shots = 8192 
max_credits = 3 
#job_exp = execute(qc, backend=backend, shots=shots, max_credits=max_credits)
from qiskit.compiler import transpile

In [2]:
#Run in real quantum computer
#Misura di tutti i qubit
n = 3
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')


job = execute(circ,
            backend=IBMQ.get_backend('ibmq_16_melbourne'),
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 000')
print(list(map(lambda c: 1/shots*c.get('000', 0), counts2)))
job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

Phi range
[0.0, 0.7853981633974483, 1.5707963267948966, 2.356194490192345, 3.141592653589793, 3.9269908169872414, 4.71238898038469, 5.497787143782138]
Job Status: job has successfully run


Counts
[{'001': 748, '101': 1190, '111': 253, '011': 167, '010': 339, '000': 2148, '100': 2869, '110': 478}, {'001': 426, '101': 216, '111': 124, '011': 149, '010': 491, '000': 3942, '100': 2533, '110': 311}, {'001': 1515, '101': 665, '111': 99, '011': 269, '010': 452, '000': 4677, '100': 434, '110': 81}, {'001': 5097, '101': 103, '111': 22, '011': 500, '010': 330, '000': 1919, '100': 188, '110': 33}, {'001': 5583, '101': 299, '111': 45, '011': 646, '010': 324, '000': 831, '100': 394, '110': 70}, {'001': 2176, '101': 243, '111': 89, '011': 387, '010': 377, '000': 2760, '100': 1948, '110': 212}, {'001': 952, '101': 2202, '111': 280, '011': 250, '010': 330, '000': 2395, '100': 1477, '110': 306}, {'001': 1258, '101': 3777, '111': 444, '011': 258, '010': 255, '000': 1249, '100': 618, '110': 333}]


Coun

In [3]:
#Run in real quantum computer - Transpile
#Misura di tutti i qubit
n = 3
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

backend = IBMQ.get_backend('ibmq_16_melbourne')
trans_ghz = transpile(circ, backend)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range],shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 000')
print(list(map(lambda c: 1/shots*c.get('000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.7853981633974483, 1.5707963267948966, 2.356194490192345, 3.141592653589793, 3.9269908169872414, 4.71238898038469, 5.497787143782138]
Job Status: job has successfully run


Counts
[{'001': 906, '101': 1288, '111': 182, '011': 130, '010': 233, '000': 2333, '100': 2843, '110': 277}, {'001': 526, '101': 328, '111': 115, '011': 127, '010': 210, '000': 4268, '100': 2462, '110': 156}, {'001': 1752, '101': 701, '111': 54, '011': 224, '010': 225, '000': 4723, '100': 457, '110': 56}, {'001': 5289, '101': 153, '111': 30, '011': 266, '010': 230, '000': 1972, '100': 232, '110': 20}, {'001': 5928, '101': 313, '111': 39, '011': 323, '010': 280, '000': 864, '100': 414, '110': 31}, {'001': 2298, '101': 324, '111': 103, '011': 257, '010': 197, '000': 2960, '100': 1948, '110': 105}, {'001': 1211, '101': 2250, '111': 155, '011': 209, '010': 220, '000': 2486, '100': 1450, '110': 211}, {'001': 1511, '101': 3853, '111': 231, '011': 178, '010': 207, '000': 1298, '100': 614, '110': 300}]


Co

In [8]:
#Run in real quantum computer - Transpile user
#Misura di tutti i qubit
n = 3
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

layout = {q[0]: 11, q[1]: 12, q[2]: 2}

backend = IBMQ.get_backend('ibmq_16_melbourne')
trans_ghz_mapped = transpile(circ, backend, initial_layout=layout)

job = execute(circ,
            backend,
            parameter_binds=[{phi: phi_val}
              for phi_val in phi_range], shots=shots)

job_monitor(job)

phi_range2 = phi_range
counts2 =[job.result().get_counts(i) for i in range(len(job.result().results))] 

print('\n')
print('Counts')
print(counts2)
print('\n')
print('Counts of 000')
print(list(map(lambda c: 1/shots*c.get('000', 0), counts2)))

job_id = job.job_id()
print('JOB ID: {}'.format(job_id))

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.7853981633974483, 1.5707963267948966, 2.356194490192345, 3.141592653589793, 3.9269908169872414, 4.71238898038469, 5.497787143782138]
Job Status: job has successfully run


Counts
[{'001': 694, '101': 354, '111': 122, '011': 257, '010': 431, '000': 6001, '100': 237, '110': 96}, {'001': 5848, '101': 279, '111': 96, '011': 333, '010': 353, '000': 991, '100': 190, '110': 102}, {'001': 2117, '101': 119, '111': 70, '011': 293, '010': 368, '000': 4974, '100': 171, '110': 80}, {'001': 2158, '101': 90, '111': 73, '011': 349, '010': 320, '000': 5042, '100': 87, '110': 73}, {'001': 6126, '101': 127, '111': 58, '011': 446, '010': 359, '000': 939, '100': 72, '110': 65}, {'001': 712, '101': 161, '111': 70, '011': 428, '010': 366, '000': 6222, '100': 148, '110': 85}, {'001': 4051, '101': 109, '111': 84, '011': 382, '010': 417, '000': 2707, '100': 347, '110': 95}, {'001': 4161, '101': 175, '111': 96, '011': 285, '010': 531, '000': 2433, '100': 383, '110': 128}]


Counts of 000
[0.732

In [7]:
n = 3
q = QuantumRegister(n, 'q')
c = ClassicalRegister(n, 'c') #n

circ = QuantumCircuit(q,c)
phi = Parameter('phi')

circ.h(q[0])

for i in range(n-1):
    circ.cx(i, i+1)
    
circ.barrier()
circ.rz(phi, range(n))
circ.barrier()

for i in reversed(range(n-1)):
    circ.cx(i, i+1)

circ.h(q[0])

circ.barrier()

for i in range(n):
    circ.measure(q[i], c[i]) 

#circ.draw(output='mpl')

print('Phi range')
phi_range = [np.pi*j/(n+1) for j in range(2*n+1+1)]
print(phi_range)
circuits = [circ.bind_parameters({phi: phi_val}) for phi_val in phi_range] 
#print(circuits[3].parameters)
#circuits[11].draw(output='mpl')

layout = {q[0]: 11, q[1]: 12, q[2]: 2}

backend = IBMQ.get_backend('ibmq_16_melbourne')
trans_ghz_mapped = transpile(circ, backend, initial_layout=layout)

trans_ghz = transpile(circ, backend)
print(circ.depth())
print(trans_ghz.depth())
print(trans_ghz_mapped.depth())

#retrieved_job = backend.retrieve_job(job_id)
#retrieved_job.result().get_counts(qc)

Phi range
[0.0, 0.7853981633974483, 1.5707963267948966, 2.356194490192345, 3.141592653589793, 3.9269908169872414, 4.71238898038469, 5.497787143782138]
8
10
8
